# Modelling and Evaluation

### Inputs
- Train set engineered CSV

### Outputs

### Objectives

- Select best hyperparameters and algorithm
- Predict CSmpA of concrete via regression

In [1]:
import os
current_dir = os.getcwd()
current_dir


'/workspace/concrete-strength/jupyter_notebooks'

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/concrete-strength'

Load data

In [5]:
import numpy as np
import pandas as pd
df = (pd.read_csv("outputs/datasets/cleaned/TrainSetEngineered.csv"))
print(df.shape)
df.head()

(824, 7)


,cement,slag,flyash,water,coarseaggregate,fineaggregate,age
0,10.083308,0.698912,1.000000,119.308050,1859.591102,74065.950376,3.461989
1,9.483185,0.698912,1.000000,115.999523,2086.779165,80994.570422,3.461989
2,8.987288,5.260016,1.000000,131.071191,1770.160564,74341.507912,3.461989
3,8.557683,0.698912,0.008382,117.379509,2065.282127,80937.293504,1.402170
4,10.184724,0.698912,1.000000,123.701476,2282.625099,55427.133259,4.681869


Create pipeline

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

def PipelineOptimization(model):
    pipeline_base = Pipeline([ ("feat_scaling", StandardScaler()),

        ("feat_selection",  SelectFromModel(model)),

        ("model", model)])
    
    return pipeline_base